In [12]:
## pip install searchtweets
from searchtweets import ResultStream, gen_rule_payload, load_credentials

In [14]:
premium_search_args = load_credentials(".twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [18]:
rule = gen_rule_payload('Mueller Report',
                        from_date="2019-04-17",
                        to_date="2019-04-19") # testing with a sandbox account
print(rule)

{"query": "Mueller Report", "toDate": "201904190000", "fromDate": "201904170000"}


In [19]:
rs = ResultStream(rule_payload=rule,
                  #max_results=10,
                  #max_pages=1,
                  **premium_search_args)

print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/fullarchive/BTG.json",
    "rule_payload": {
        "query": "Mueller Report",
        "toDate": "201904190000",
        "fromDate": "201904170000"
    },
    "tweetify": true,
    "max_results": 500
}


In [20]:
## makes 1 request every time you call this (limit 50 requests/month)
tweets = list(rs.stream())

In [186]:
## "Mueller Report" from 4-17-2019 to 4-19-2019

import copy
p_tweets = copy.deepcopy(tweets)

file1 = open("mueller.csv", "w+", encoding="utf-8")
headers1 = ['created_at, id, id_str, text, source, truncated, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str, in_reply_to_screen_name, user_id, user_screen_name, geo, coordinates, place, contributors, retweeted_id, is_quote_status, quote_count, reply_count, retweet_count, favorite_count, entities, favorited, retweeted, filter_level, lang, matching_rules']
file1.write("{}\n".format(','.join(header for header in headers1))) 

file2 = open("mueller_users.csv", "w", encoding="utf-8") 
headers2 = ['id, id_str, name, screen_name, location, url, description, translator_type, protected, verified, followers_count, friends_count, listed_count, favourites_count, statuses_count, created_at, utc_offset, time_zone, geo_enabled, lang, contributors_enabled, is_translator, profile_background_color, profile_background_image_url, profile_background_image_url_https, profile_background_tile, profile_link_color, profile_sidebar_border_color, profile_sidebar_fill_color, profile_text_color, profile_use_background_image, profile_image_url, profile_image_url_https, profile_banner_url, default_profile, default_profile_image, following, follow_request_sent, notifications'] 
file2.write("{}\n".format(','.join(header for header in headers2))) 

file3 = open("mueller_retweets.csv", "w", encoding="utf-8") 
headers3 = ['created_at, id, id_str, text, source, truncated, in_reply_to_status_id, in_reply_to_status_id_str, in_reply_to_user_id, in_reply_to_user_id_str, in_reply_to_screen_name, user, geo, coordinates, place, contributors, is_quote_status, extended_tweet, quote_count, reply_count, retweet_count, favorite_count, entities, favorited, retweeted, filter_level, lang'] 
file3.write("{}\n".format(','.join(header for header in headers3))) 

for i in range(len(p_tweets)): 
    params = list(p_tweets[i].values())
    if(i in [22,38,44,45,51,57,62,77,88,144,170,182,187,210,237,239,248,270,274,277,314,334,362,424,425,430,476,494]):
        continue
#         t_params = params[:11] + params[12:] 
#         params = t_params
    fields = params[:11]

    user_id = params[11]['id'] 
    user_screen_name = params[11]['screen_name']
    fields.extend([user_id,user_screen_name]) 
    user_fields = list(list(params[11].values()))
    
    fields += params[12:16] 
    if(params[16] == False):
        fields.append(None)
        retweet_fields = []*27
    else:
        fields.append(params[16]['id'])
        retweet_fields = list(list(params[16].values()))
    
    fields += params[17:]
    
    file1.write("{}\n".format(','.join(str(field) for field in fields)))
    file2.write("{}\n".format(','.join(str(u_field) for u_field in user_fields)))
    file3.write("{}\n".format(','.join(str(r_field) for r_field in retweet_fields)))
